In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
Real_estate=pd.read_csv('data_update.csv')

In [ ]:
pd.DataFrame(Real_estate)

In [ ]:
Real_estate.rename(columns=str.lower,inplace=True)

In [ ]:
Real_estate.homeid.nunique()

In [ ]:
Real_estate.solutionid.nunique()

In [ ]:
Real_estate['salestate']=Real_estate['salestate'].fillna("未公开")
Real_estate['mainstockrate']=Real_estate['mainstockrate'].fillna(0)
Real_estate['totalbuildarea']=Real_estate['totalbuildarea'].fillna(0)
Real_estate['supplyarea']=Real_estate['supplyarea'].fillna(0)
Real_estate['tradearea']=Real_estate['tradearea'].fillna(0)
Real_estate['totaltradeprice']=Real_estate['totaltradeprice'].fillna(0)
Real_estate['sumsupplyarea']=Real_estate['sumsupplyarea'].fillna(0)

In [ ]:
print(Real_estate.info())

In [ ]:
cat_columns=["salestate","roomusage"]
for column in cat_columns:
    Real_estate[column]=Real_estate[column].astype('category')

In [ ]:
print(Real_estate['salestate'].value_counts())

In [ ]:
Real_estate['salestate']=Real_estate['salestate'].map({"售罄":1,"预售":0,"销售":0,"尾盘":0,"形象":0,"未公开":0,"烂尾楼":0})

In [ ]:
Real_estate1=Real_estate.copy()
Real_estate1.sort_values(["solutionid","month"],inplace=True)
Real_estate1=Real_estate1.drop_duplicates(subset=['month','homeid','roomusage'],keep="last")
Real_estate1

In [ ]:
print(Real_estate1.info())

In [ ]:
print(Real_estate1['roomusage'].value_counts())

In [ ]:
Real_estate1.homeid.nunique()

In [ ]:
Real_estate1['roomusage']=Real_estate['roomusage'].map({"其他":1,'普通住宅':0,"商业":0,"别墅":0,'酒店式公寓':0,'办公':0,"动迁安置房":0,"工业":0,"限价房":0,"配套商品房":0})

In [ ]:
Totalprice=Real_estate1["mainstockrate"]*Real_estate1["itemstockrate"]*Real_estate1["totaltradeprice"]*0.0001

In [ ]:
Real_estate1["Totalprice"]=Totalprice
Real_estate1

In [ ]:
pt1=Real_estate1.pivot_table(index='homeid',columns="roomusage",values="tradearea",aggfunc=np.sum)
print(pt1)

In [ ]:
df1=pt1[[0,1]]

In [ ]:
Real_estate2=Real_estate1.copy()
Real_estate2.sort_values(["homeid","month"],inplace=True)
Real_estate2=Real_estate2.drop_duplicates(subset=["homeid"],keep="last")
Real_estate2.set_index("homeid",inplace=True)
# 将“其他”的累计tradearea计算出来

In [ ]:
Real_estate2["Nettotaltradearea"]=(Real_estate2["totaltradearea"].fillna(0))-(df1[1].fillna(0))

# 从totaltradearea 中扣除“其他”traderea"

In [ ]:
pd.DataFrame(Real_estate2)

In [ ]:
#TEST, DELETE LATER
Real_estate2.loc["6d5790f8-3a8d-4079-b2ba-4eae0cda87b4",:]

In [ ]:
Real_estate2.reset_index(inplace=True)

In [ ]:
df2=Real_estate2[['solutionid','homeid','maincaption','Totalprice','Nettotaltradearea','salestate','totalbuildarea','sumsupplyarea','month']]
df2.head()

In [ ]:
def calc_weights(the_area):
    return the_area/the_area.sum()
weights1=df2.groupby('solutionid').totalbuildarea.apply(calc_weights)
weights1

In [ ]:
df2["weights1"]=weights1
df2.head()

In [ ]:
sumsupplyarea1=df2.loc[:,"sumsupplyarea"]*df2.loc[:,"weights1"]

In [ ]:
df2["sumsupplyarea1"]=sumsupplyarea1

In [ ]:
df2['Totalbuildarea1']=df2.loc[:,['totalbuildarea','sumsupplyarea1']].apply(max,axis=1)
df2

In [ ]:
df2.set_index(["solutionid","salestate"],inplace=True)
df2

In [ ]:
df4=df2.groupby(['solutionid','salestate']).sum()
df4
# Only use the totalbuildarea column

In [ ]:
df3=df2.groupby(['solutionid','salestate']).mean()
df3=df3.xs(1,level='salestate')
# only use "Nettotaltradearea"

In [ ]:
df4=df4.xs(1,level='salestate')
df4=df4.drop(['Totalprice','Nettotaltradearea','sumsupplyarea','weights1','sumsupplyarea1',"Totalbuildarea1"],axis=1)
df4

In [ ]:
#TEST, DELETE LATER
df3.loc["05759d0c-5fa5-433a-b16d-4fcd07ead827",:]

In [ ]:
Nettotaltradearea1=df3['Nettotaltradearea']-df4['totalbuildarea']
df3["Nettotaltradearea1"]=Nettotaltradearea1

In [ ]:
df3.reset_index(inplace=True)
df3

In [ ]:
df3=df3.loc[:,['solutionid',"Nettotaltradearea1"]]
df3

In [ ]:
df3.set_index(["solutionid"],inplace=True)
df3

In [ ]:
df5=df2.xs(0,level='salestate')
df5

In [ ]:
df6=pd.merge(df5,df3,on=['solutionid'],how='left')
df6.reset_index(inplace=True)
df6

In [ ]:
df6['Nettotaltradearea2']=df6['Nettotaltradearea1'].fillna(df6['Nettotaltradearea'])
df6.head()

In [ ]:
df6['Nettotaltradearea3']=df6.apply(lambda x:0 if x['Nettotaltradearea2']<0 else x['Nettotaltradearea2'],axis=1)

In [ ]:
def calc_weights(the_area):
    return the_area/the_area.sum()
weights=df6.groupby('solutionid').totalbuildarea.apply(calc_weights)
weights

In [ ]:
df6["weights"]=weights
df6.head()

In [ ]:
Totalvalue=df6.loc[:,"Totalprice"]*(df6.loc[:,"Totalbuildarea1"]-df6.loc[:,"Nettotaltradearea3"]*df6.loc[:,"weights"])

In [ ]:
df6["Totalvalue"]=Totalvalue
df6['Totalvalue1']=df6.apply(lambda x:0 if x['Totalvalue']<0 else x['Totalvalue'],axis=1)

In [ ]:
pt2=df6.pivot_table(index=['maincaption'],values="Totalvalue1",aggfunc=np.sum)
pt2

In [ ]:
pt2.to_excel("销售统计6.xls")